In [ ]:
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import os
import json
import cv2

In [ ]:
TRAIN_PATH = "/kaggle/input/brain-tumor-image-dataset-semantic-segmentation/train"
VALID_PATH = "/kaggle/input/brain-tumor-image-dataset-semantic-segmentation/valid"
TEST_PATH = "/kaggle/input/brain-tumor-image-dataset-semantic-segmentation/test"

In [ ]:

name = []
annotation_file = ""

for dirname, _, filenames in os.walk(TRAIN_PATH):
    for filename in filenames:
        if filename.endswith(".jpg"):
            name.append(filename)
        else:
            annotation_file = TRAIN_PATH + "/" + filename

In [ ]:
df = pd.DataFrame({"id": name}, index = np.arange(0, len(name)))
print("total images", len(df))

In [ ]:
with open(annotation_file, "r") as f:
    ann_json = json.loads(f.read())
    print(ann_json.keys())
    print(ann_json["images"][0])
    print(ann_json["annotations"][0])

In [ ]:
def get_id(filename):
    for image in ann_json["images"]:
        if image["file_name"] == filename:
            return image["id"]

In [ ]:
filename = df["id"][0]
print(filename)

img_id = get_id(filename)
print(img_id)

def get_bbox(img_id):
    for ann in ann_json["annotations"]:
        if ann["image_id"] == img_id:
            return ann["bbox"]
print(get_bbox(img_id))
bbox = get_bbox(img_id)
x, y, width, height = bbox

pt1 = (int(x), int(y))
pt2 = (int(x) + int(width), int(y) + int(height))
color = (255, 255, 255)

img = Image.open(TRAIN_PATH + "/" + filename)
img_cv = np.array(img)

thickness = 2

cv2.rectangle(img_cv, pt1, pt2, color, thickness)
plt.imshow(cv2.cvtColor(img_cv, cv2.COLOR_BGR2RGB))
plt.axis("off")